<a href="https://colab.research.google.com/github/mareloraby/Sentiment-Arcs/blob/main/Hedonometer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


!pip install tensorflow
import os
import re
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

from IPython.display import clear_output
clear_output()

In [ ]:
def split3(df):
  pos = df[df['label'] == 2]
  neg = df[df['label'] == 0]
  neu = df[df['label'] == 1]
  return neg,neu,pos


def normalizeRange(x,a,b):
  normalized = (b-a) * ((x-np.min(x))/(np.max(x)-np.min(x))) + a #range
  return normalized
 
def normalizeScoreadder(df):

  neg0,neu0,pos0 = split3(df)

  pos0['score_norm'] = normalizeRange(pos0['norm01'], 0 , 1)  
  neu0['score_norm'] = normalizeRange(neu0.norm01, 0,0)  
  neg0['score_norm'] = normalizeRange(neg0.norm01,0,-1)  


  appended = pos0.append(neu0,ignore_index=True).append(neg0,ignore_index = True)
  appended.sort_values('Datetime',ignore_index=True, inplace = True)

  return appended


In [ ]:
allTweets = pd.read_csv("/content/drive/MyDrive/NLP/COVID19data/allTweetstilljune21100.csv",  lineterminator='\n') 

allTweets[['date', 'time']] = allTweets['Datetime'].str.split(' ', 1, expand=True)
allTweets['date'] = pd.to_datetime(allTweets['date'])
allTweets2= normalizeScoreadder(allTweets)
allTweets = allTweets2

clear_output()
del allTweets['index']
allTweets

,Datetime,full_text,label,norm01,score_norm,date,time
0,2020-02-01 00:00:00,Pointing to the outbreak of the coronavirus in...,0,0.655362,-0.489042,2020-02-01,00:00:00
1,2020-02-01 00:00:28,Dalton man wears face mask after coronavirus i...,1,0.882608,0.000000,2020-02-01,00:00:28
2,2020-02-01 00:00:35,Coronavirus lies are rife on social media. Her...,0,0.941583,-0.929058,2020-02-01,00:00:35
3,2020-02-01 00:01:01,@CassandraRules I think its due to their coron...,0,0.980004,-0.988123,2020-02-01,00:01:01
4,2020-02-01 00:01:34,6/The truth is that the OVERWHELMING majority ...,0,0.951164,-0.943787,2020-02-01,00:01:34
...,...,...,...,...,...,...,...
1500710,2021-06-30 23:56:00,TA CAINDO FML https://t.co/O3SKpPupc4,1,0.822455,0.000000,2021-06-30,23:56:00
1500711,2021-06-30 23:56:25,@ShuttleMrGeo Not true.\nIt has been isolated....,0,0.753796,-0.640367,2021-06-30,23:56:25
1500712,2021-06-30 23:56:41,What are the odds of dying form COVID19 Vs sta...,0,0.933541,-0.916695,2021-06-30,23:56:41
1500713,2021-06-30 23:56:59,Women's welfare &amp; empowerment is non-negot...,0,0.568037,-0.354792,2021-06-30,23:56:59


In [ ]:
temp1 = allTweets.loc[(allTweets['date'] == '2020-02-26')]
temp1
del temp1

In [ ]:

!pip install labMTsimple
!pip install marisa_trie

from labMTsimple.storyLab import *
from labMTsimple.speedy import *

!git clone https://github.com/andyreagan/labMT-simple.git

lang = 'english'
labMT,labMTvector,labMTwordList = emotionFileReader(stopval=0.0,lang=lang,returnVector=True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.7 MB 14.4 MB/s 
  Created wheel for labMTsimple: filename=labMTsimple-2.8.7-py3-none-any.whl size=3792665 sha256=060ced22e43861034453fd1e1e978c79e7a9c7bcb1a338c448951e6b2d89bac8
  Stored in directory: /root/.cache/pip/wheels/a8/8e/e9/8497318bfad7a7b87dbf6dbb4414de7d44cd3fdec53f2cf87e
Successfully built labMTsimple
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 14.1 MB/s 
Cloning into 'labMT-simple'...
remote: Enumerating objects: 2162, done.
remote: Total 2162 (delta 0), reused 0 (delta 0), pack-reused 2162
Receiving objects: 100% (2162/2162), 198.91 MiB | 25.83 MiB/s, done.
Resolving deltas: 100% (952/952), done.
Checking out files: 100% (630/630), done.


In [ ]:
hedonometer_results = pd.DataFrame(columns = ['date', 'h_score'])

In [ ]:
from datetime import date, timedelta

start_date = date(2020, 6, 1)
end_date = date(2020, 11, 1)

delta = timedelta(days=1)

while start_date < end_date:

    print(start_date.strftime("%Y-%m-%d"))
    temp = allTweets.loc[(allTweets['date'] == start_date.strftime("%Y-%m-%d"))][0:5501]
    full_string = (temp['full_text'].to_string(index = False))
    del temp   

    _,saturdayFvec = emotion(full_string,labMT,shift=True,happsList=labMTvector)
    del full_string

    saturdayStoppedVec = stopper(saturdayFvec,labMTvector,labMTwordList,stopVal=1.0)
    saturdayValence = emotionV(saturdayStoppedVec,labMTvector)

    # print(saturdayValence)
    hedonometer_results = hedonometer_results.append({'date' :  start_date.strftime("%Y-%m-%d"), 'h_score' : saturdayValence},
                                                     ignore_index = True)

    start_date += delta

In [ ]:
hedonometer_results

,date,h_score
0,2020-06-01,5.816380
1,2020-06-02,5.792770
2,2020-06-03,5.842473
3,2020-06-04,5.812790
4,2020-06-05,5.764552
...,...,...
148,2020-10-27,5.720597
149,2020-10-28,5.772871
150,2020-10-29,5.788769
151,2020-10-30,5.774601


In [ ]:
del hedonometer_results['index']

In [ ]:
# hedonometer_results.to_csv('/content/drive/MyDrive/NLP/COVID19data/hedonometer_results_111220.csv')
# hedonometer_results.to_csv('/content/drive/MyDrive/NLP/COVID19data/hedonometer_results_040520.csv')
# hedonometer_results.to_csv('/content/drive/MyDrive/NLP/COVID19data/hedonometer_results_01020321.csv')
# hedonometer_results.to_csv('/content/drive/MyDrive/NLP/COVID19data/hedonometer_results_060708091021.csv')